In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [3]:
cd 'drive/My Drive/Colab Notebooks/challenge-matrix/Challenge 2/'

/content/drive/My Drive/Colab Notebooks/challenge-matrix/Challenge 2


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#from sklearn.dummy import DummyRegressor
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.ensemble import RandomForestRegressor


import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mea
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from hyperopt import hp, fmin, tpe, STATUS_OK


In [0]:
df = pd.read_hdf('data/car.h5')

In [0]:
Suff_Cat = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue
  fac_val = df[feat].factorize()[0]

  if (Suff_Cat in feat):
    df[feat] = fac_val
  else:
    df [feat + Suff_Cat] = fac_val

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else x.split(' ')[0])

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ','')))

In [0]:
param = {
    'max_depth':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed':0

}

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df ['price_value'].values


  scores = cross_val_score(model, X, y, cv=3,scoring="neg_mean_absolute_error")
  return np.mean(scores), np.std(scores)

In [0]:
feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc','param_marka-pojazdu__cat','param_typ__cat','feature_kamera-cofania__cat','param_pojemność-skokowa','seller_name__cat','param_kod-silnika__cat','param_model-pojazdu__cat','feature_wspomaganie-kierownicy__cat','param_wersja__cat']

In [16]:
run_model(xgb.XGBRegressor(**param) , feats)

[12:12:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:12:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:12:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9652.152829006945, 69.42030265847033)

In [0]:
def obj_func(params):
  print("Training with params: ", params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**param) , feats)
  return {'loss':np.abs(mean_mae), 'status':STATUS_OK}

  

In [0]:
xgb_reg_params = {
    'lerning_rate': hp.choice('lerning_rate', np.arange(0.05, 0.31, 0.05)),
    'max_depth': hp.choice('max_depth', np.arange(6, 16, 1, dtype=int)),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective': 'reg:squarederror',
    'n_estimators':100,
    'seed':0,
}

In [20]:
best = fmin (obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

Training with params: 
{'colsample_bytree': 0.65, 'lerning_rate': 0.15000000000000002, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8}
[12:39:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:40:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:40:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Training with params: 
{'colsample_bytree': 0.65, 'lerning_rate': 0.05, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001}
[12:40:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:40:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:square